In [1]:
!pip install torch>=1.7.1
!pip install transformers
!pip install torchvision

In [ ]:
# !pip install tensorflow
# !pip install setuptools==69.5.1
# !pip install --upgrade pip setuptools wheel
# !pip install numpy --only-binary :all

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
# import gluonnlp as nlp
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import scipy.sparse as sp

In [ ]:
import tensorflow as tf
tf.__version__

'2.10.0'

In [2]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
print(device)

cuda


In [3]:
file_path = '/content/target_df.xlsx'
# file_path = "D:\\sample\\content\\신장내과_본원_외래_20230701_20231231_filtered.xlsx"

# selected_columns = ['ICD10', 'transstr']
selected_columns = ['ICD10', 'CC', 'PI', 'CC_trans', 'PI_trans', 'record']
df = pd.read_excel(file_path, usecols=selected_columns, dtype=str)

print('전체 데이터 수 :', len(df))
df.head(5)

전체 데이터 수 : 826


,ICD10,CC,PI,CC_trans,PI_trans,record
0,N391,NaN,115-61-61_x000D_\n2023.5.10._x000D_\n_x000D_\n...,NaN,115-61-61 2023.5.10.Protein Protein Protein Pr...,115-61-61 2023.5.10.Protein Protein Protein Pr...
1,N184,"CKD4, Cr 2.26, eGFR 29ml/min","130/61/69_x000D_\n요증상(-), edema(-), 요독증상(-), D...","ckd4, cr 2.26, egfr 29ml/min","130/61/69 There is no symptom, no edema, no dy...","ckd4, cr 2.26, egfr 29ml/min. 130/61/69 There ..."
2,N184,CKD4,"153/75/85_x000D_\n요증상(-), edema(-), 요독증상(-), D...",ckd4,"153/75/85 There is no symptom, no edema, no di...","ckd4. 153/75/85 There is no symptom, no edema,..."
3,N185,"Cr 3.74, eGFR 13.56","148/90/64_x000D_\n요증상(-), edema mild_x000D_\nC...","cr 3.74, egfr 13.56","148/90/64 No symptoms, edema Mild CKD D/T ADPK...","cr 3.74, egfr 13.56. 148/90/64 No symptoms, ed..."
4,N183,lab abnormality,선생님 안녕하십니까_x000D_\n_x000D_\n상기 병명으로 입원하여 치료중인 ...,lab abnormality,I am hospitalized and treated with a disease n...,lab abnormality. I am hospitalized and treated...


In [ ]:
# 전처리 1
# df['CC_2'] = df['CC'].str.replace("_x000D_\n", "\n")
# df['CC_2'] = df['CC_2'].str.replace("_x000D_\n", "\n")
# df['CC_2'] = df['CC_2'].str.replace("(+)", " 있음")
# df['CC_2'] = df['CC_2'].str.replace("(-))", " 없음")
# df['CC_2'] = df['CC_2'].str.strip()

# df['PI_2'] = df['PI'].str.replace("_x000D_\n", "\n")
# df['PI_2'] = df['PI_2'].str.replace("(+)", " 있음")
# df['PI_2'] = df['PI_2'].str.replace("(-)", " 없음")
# df['PI_2'] = df['PI_2'].str.strip()

# # df['HX_2'] = df['HX'].str.replace("_x000D_\n", " ")
# # df['HX_2'] = df['HX_2'].str.strip()

# df.head(5)

In [4]:
# stopwords dataset load
# sw_path = "D:\\sample\\content2\\stopwords.xlsx"
sw_path = "/content/stopwords.xlsx"
sw_df = pd.read_excel(sw_path)
for i in range(len(sw_df)) :
  stop_words = sw_df['stop words'].tolist()
# print(stop_words)

In [5]:
def remove_nonAscii(text):
    rtn = ''
    if not pd.isna(text):
        rtn = "".join(str for str in text if ord(str)<128)
    return rtn

def make_lower_case(text):
    rtn = ''
    if not pd.isna(text):
        rtn = text.lower()
    return rtn

def remove_stop_words(text):
    rtn = ''
    if not pd.isna(text):
        rtn = text.split()
        rtn = [w for w in rtn if not w in stop_words]
        rtn = " ".join(rtn)
    return rtn

def remove_punctuation(text):
    rtn = ''
    if not pd.isna(text):
        tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
        rtn = tokenizer.tokenize(text)
        rtn = " ".join(rtn)
    return rtn

def remove_others(text):
    rtn = ''
    if not pd.isna(text):
        s_text = text.split()
        s_list = []

        for w in s_text:
            if len(w) > 1:
                s_list.append(w)
        rtn = " ".join(s_list)
    return rtn

In [ ]:
# # df['CC_2'] = df['CC'].apply(remove_nonAscii)
# df['CC_2'] = df['CC_2'].apply(make_lower_case)
# df['CC_2'] = df['CC_2'].apply(remove_stop_words)
# # df['CC_2'] = df['CC_2'].apply(remove_punctuation)
# df['CC_2'] = df['CC_2'].apply(remove_others)

# # df['PI_2'] = df['PI'].apply(remove_nonAscii)
# df['PI_2'] = df['PI_2'].apply(make_lower_case)
# df['PI_2'] = df['PI_2'].apply(remove_stop_words)
# # df['PI_2'] = df['PI_2'].apply(remove_punctuation)
# df['PI_2'] = df['PI_2'].apply(remove_others)

In [ ]:
# pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 번역
# from googletrans import Translator

# def trans_hng_to_eng(text):
#     translator = Translator()
#     if not text:  # None, 빈 문자열 모두 체크
#         return None
#     try:
#         translated = translator.translate(text, dest='en')
#         return translated.text
#     except Exception as e:
#         print(f"Translation error: {e}")
#         return text

In [ ]:
# trans_hng_to_eng(df['주호소'][1])

'CKD4, Cr 2.26, eGFR 29ml/min'

In [ ]:
# df['transstr'] = df['현병력'].apply(trans_hng_to_eng)
# df['transstr'][:5]

# df['CC_trans'] = df['주호소'].apply(trans_hng_to_eng)
# df['CC_trans'][:5]

0                                
1    CKD4, Cr 2.26, eGFR 29ml/min
2                            CKD4
3             Cr 3.74, eGFR 13.56
4                 lab abnormality
Name: CC_trans, dtype: object

In [ ]:
# def combine_columns(col_1, col_2):
#     result = ''

#     if col_1:
#         result = str(col_1).strip() + '.'

#     if col_2:
#         result += ' ' + str(col_2).strip()

#     return result.strip()

# df["record"] = df.apply(lambda x: combine_columns(x['CC_trans'], x['PI_trans']), axis=1)

In [ ]:
# df.apply
# df['record'].head(10)
df

,ICD10,주호소,PI_trans,CC_trans,record
0,N391,NaN,115-61-61 2023.5.10.Suspicion of proteinuria i...,,115-61-61 2023.5.10.Suspicion of proteinuria ...
1,N184,"CKD4, Cr 2.26, eGFR 29ml/min","130/61/69 Symptoms (-), edema (-), urbanic sym...","CKD4, Cr 2.26, eGFR 29ml/min","CKD4, Cr 2.26, eGFR 29ml/min 130/61/69 Symptom..."
2,N184,CKD4,"153/75/85 osophy (-), edema (-), urinary sympt...",CKD4,"CKD4 153/75/85 osophy (-), edema (-), urinary ..."
3,N185,"Cr 3.74, eGFR 13.56",148/90/64 Idema Mild CKD D/T ADPKD has been tr...,"Cr 3.74, eGFR 13.56","Cr 3.74, eGFR 13.56 148/90/64 Idema Mild CKD D..."
4,N183,lab abnormality,"Hello teacher, I am a patient who is being hos...",lab abnormality,"lab abnormality Hello teacher, I am a patient ..."
...,...,...,...,...,...
821,N183,Azotemia,136/73-73 23.02 EDEMA occurs after shoulder su...,Azotemia,Azotemia 136/73-73 23.02 EDEMA occurs after sh...
822,N184,Azotemia,The patient is a diagnostic [DJD] patient of 8...,Azotemia,Azotemia The patient is a diagnostic [DJD] pat...
823,N185,konos 등록,"KONOS Registration HD 3 times a week, gold at ...",KONOS registration,KONOS registration KONOS Registration HD 3 tim...
824,R601,Generalized edema,He has been edema two months ago.Urine comes o...,Generalized edema,Generalized edema He has been edema two months...


In [16]:
# 학습용, 테스트용, 검증용 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(df['record'], df['ICD10'], test_size=0.3, shuffle=True, random_state=52, stratify=df['ICD10'])
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=True, random_state=73, stratify=y_temp)

# X_train, X_temp, y_train, y_temp = train_test_split(df['record'], df['ICD10'], test_size=0.2, random_state=None, stratify=df['ICD10'])
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(X_train)
print("=======================================\n")
print(X_test)
print("=======================================\n")
print(X_val)
print("=======================================\n")
print(y_train)
print("=======================================\n")
print(y_test)
print("=======================================\n")
print(y_val)

431    azotemia. The patient is a patient of 84 years...
263    azotemia. I have never checked it before 2023....
786    azotemia. Four years ago, HTN was confirmed in...
331    hematuria. Hematuria Clot occurred from HTN, D...
760    proteinuria. No HTN, no DM No MDS F/U In the t...
                             ...                        
258    112-56-78 Vomiting of pruritus vomiting vsd Cl...
58     hematuria. Hematuria is heard from the 20s and...
518    hematuria. 2023. Listen to hematuria.Gross Hem...
417    2016년 cr 1.9, 2023.6 cr 1.6mg/dl. 139/88/88 No...
163                                            121-65-91
Name: record, Length: 578, dtype: object

161    proteinuria. OLD ICH, RT BG HTN (I knew it bef...
30     for evaluation persistent proteinuria. From ar...
20     116-72-67 Hypertension 4-year Gallens Cypse Pi...
766    130-84-103 dm dm retinopathy (avastin laser) 2...
39     Reminery decrease. 2017. CR 0.89 EGFR 85 U/A P...
                             ...              

In [17]:
### datasets 분할 확인
y_train_counts =  y_train.value_counts()
y_test_counts = y_test.value_counts()
y_val_counts = y_val.value_counts()

y_train_counts_df = y_train_counts.reset_index()
y_train_counts_df.columns = ['ICD10', 'y_train_count']
y_test_counts_df = y_test_counts.reset_index()
y_test_counts_df.columns = ['ICD10', 'y_test_count']
y_val_counts_df = y_val_counts.reset_index()
y_val_counts_df.columns = ['ICD10', 'y_val_count']

# 두 DataFrame을 병합
counts_df = pd.merge(y_train_counts_df, y_test_counts_df, on='ICD10', how='outer').fillna(0)
counts_df = pd.merge(counts_df, y_val_counts_df, on='ICD10', how='outer').fillna(0)

# 정수형으로 변환 (NaN 처리 후)
counts_df['y_train_count'] = counts_df['y_train_count'].astype(int)
counts_df['y_test_count'] = counts_df['y_test_count'].astype(int)
counts_df['y_val_count'] = counts_df['y_val_count'].astype(int)

print(counts_df)
### End of datasets 분할 확인

   ICD10  y_train_count  y_test_count  y_val_count
0   N183            197            43           42
1   N185             74            16           16
2   N184             43             9            9
3   N391             41             9            9
4   N178             39             8            9
5   R311             36             8            8
6   N029             29             7            6
7   R798             24             6            5
8   Z524             22             4            5
9   N182             20             4            4
10  N059             19             4            4
11  N028             12             2            3
12  R808             11             2            2
13  R601             11             2            2


In [18]:
# Categorical encoding
label_dict = df['ICD10'].unique()
print("labels :", label_dict)
category = label_dict.reshape(-1, 1)

# label one-hot-encoding
onehot_encoder = OneHotEncoder() #sparse_output=False

# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
y_train_encoded = onehot_encoder.fit_transform(pd.DataFrame(y_train)[['ICD10']])
y_test_encoded = onehot_encoder.transform(pd.DataFrame(y_test)[['ICD10']])
y_val_encoded = onehot_encoder.transform(pd.DataFrame(y_val)[['ICD10']])

print(y_train_encoded.toarray())

labels : ['N391' 'N184' 'N185' 'N183' 'N029' 'N182' 'N178' 'R798' 'R808' 'R311'
 'N059' 'Z524' 'N028' 'R601']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [14]:
len(label_dict)

14

In [13]:
onehot_encoder.categories_

[array(['N028', 'N029', 'N059', 'N178', 'N182', 'N183', 'N184', 'N185',
        'N391', 'R311', 'R601', 'R798', 'R808', 'Z524'], dtype=object)]

In [14]:
onehot_encoder.inverse_transform(y_train_encoded)

array([['N183'],
       ['N178'],
       ['N183'],
       ['N029'],
       ['R808'],
       ['N391'],
       ['N184'],
       ['N183'],
       ['N183'],
       ['N183'],
       ['N183'],
       ['N183'],
       ['N185'],
       ['N185'],
       ['N183'],
       ['N183'],
       ['N185'],
       ['R311'],
       ['R798'],
       ['N185'],
       ['N183'],
       ['N184'],
       ['N183'],
       ['N183'],
       ['N391'],
       ['N059'],
       ['N183'],
       ['N183'],
       ['N183'],
       ['R808'],
       ['N028'],
       ['R601'],
       ['N178'],
       ['N184'],
       ['N178'],
       ['N182'],
       ['N391'],
       ['N185'],
       ['N185'],
       ['N183'],
       ['N182'],
       ['N183'],
       ['Z524'],
       ['Z524'],
       ['N185'],
       ['N183'],
       ['N185'],
       ['N183'],
       ['N183'],
       ['N029'],
       ['R311'],
       ['N183'],
       ['Z524'],
       ['N183'],
       ['R311'],
       ['N183'],
       ['N185'],
       ['N183'],
       ['N183'

In [19]:
y_train_dense = y_train_encoded.todense()
y_test_dense = y_test_encoded.todense()
y_val_dense = y_val_encoded.todense()

In [ ]:
#https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [ ]:
class EMRDiagcdDataset(Dataset):

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        record = self.df.iloc[index]['record']
        # diagcd = self.df.iloc[index]['ICD10']
        diagcd = self.df.iloc[index]['encoded_ICD10']

        return record, diagcd

In [ ]:
class EMRDiagcdDataset2(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [33]:
class EMRDiagcdDataset3(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, padding=True, truncation=True, max_length=self.max_length, return_tensors='pt')
        return encoding, label

In [38]:
def train_model(model, train_loader, optimizer, num_epochs):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            labels = torch.tensor(labels, dtype=torch.float32)
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs['input_ids'].size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

In [12]:
class UMLSBertCNN(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(UMLSBertCNN, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.cnn = nn.Conv1d(in_channels=self.bert.config.hidden_size, out_channels=128, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Linear(128 * (self.bert.config.max_position_embeddings // 2), num_labels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        last_hidden_state = last_hidden_state.permute(0, 2, 1)  # (batch_size, hidden_size, seq_len)
        cnn_output = self.cnn(last_hidden_state)  # (batch_size, 128, seq_len)
        cnn_output = self.relu(cnn_output)
        cnn_output = self.pool(cnn_output)  # (batch_size, 128, seq_len//2)
        cnn_output = cnn_output.view(cnn_output.size(0), -1)  # flatten
        logits = self.fc(cnn_output)  # (batch_size, num_labels)
        output = self.sigmoid(logits)  # For multi-label classification
        return output

In [30]:
max_len = 0

for rowstr in df['record']:
    if len(rowstr) > max_len:
        max_len = len(rowstr)

print(max_len)

1060


In [24]:
# Load pre-trained tokenizer & model
# bert_model_name = "GanjinZero/UMLSBert_ENG"
bert_model_name = "medicalai/ClinicalBERT"
num_labels = len(label_dict)
max_length = max_len * 2

# model = UMLSBertCNN(bert_model_name, num_labels)
# tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# Model
# model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=num_labels)
model = AutoModel.from_pretrained(bert_model_name, num_labels=num_labels)
model.to(device)

# Vocabulary
# vocab = tokenizer.get_vocab()

# Tokenize
# tok = tokenizer.tokenize

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GanjinZero/UMLSBert_ENG and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [36]:
# Create Dataset and DataLoader
train_dataset = EMRDiagcdDataset3(X_train, y_train_dense, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=2)

test_dataset = EMRDiagcdDataset3(X_test, y_test_dense, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=2)

In [39]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
num_epochs = 1

model.train()
train_model(model, train_loader, optimizer, num_epochs)

RuntimeError: stack expects each tensor to be equal size, but got [1, 25] at entry 0 and [1, 71] at entry 1

In [ ]:
# label encoding
# unique_label = df['ICD10'].unique()
# print("labels :", unique_label)

# label_encoder = LabelEncoder()
# label_encoder.fit(unique_label)

# # Training dataset labels
# temp_df = Y_train.to_frame()
# temp_df['encoded_ICD10'] = label_encoder.transform(temp_df['ICD10'])
# Y_train_encoding = pd.Series(temp_df['encoded_ICD10'])

# # Test dataset labels
# temp_df = Y_test.to_frame()
# temp_df['encoded_ICD10'] = label_encoder.transform(temp_df['ICD10'])
# Y_test_encoding = pd.Series(temp_df['encoded_ICD10'])

labels : ['N391' 'N184' 'N185' 'N183' 'N029' 'N182' 'N178' 'R798' 'R808' 'R311'
 'N059' 'Z524' 'N028' 'R601']


In [ ]:
max_len = 2048
batch_size = 32
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# Training data loader
train_dataset = EMRDiagcdDataset2(X_train, 0, 1, tok, vocab, max_len, True, False)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_worker=5)

# Test data loader
test_dataset = EMRDiagcdDataset2(X_test, 0, 1, tok, vocab, max_len, True, False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=false, num_worker=5)

NameError: name 'nlp' is not defined

In [ ]:
# Training data loader
train_df = pd.concat([X_train, Y_train_encoding, Y_train], axis=1)
train_dataset = EMRDiagcdDataset2(train_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test data loader
test_df = pd.concat([X_test, Y_test_encoding, Y_test], axis=1)
test_dataset = EMRDiagcdDataset2(test_df)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=false)

In [ ]:
# train_df.to_excel('D:\\sample\\content\\train_df.xlsx')
df.to_excel('D:\\sample\\content\\translated_df.xlsx')

In [ ]:
# class UmlsBERTClassifier(nn.Module):
#     def __init__(self,
#                  bert,
#                  hidden_size = 768,
#                  num_classes=14,   ##클래스 수 조정##
#                  dr_rate=None,
#                  params=None):
#         super(UmlsBERTClassifier, self).__init__()
#         self.bert = bert
#         self.dr_rate = dr_rate

#         self.classifier = nn.Linear(hidden_size , num_classes)
#         if dr_rate:
#             self.dropout = nn.Dropout(p=dr_rate)

#     def gen_attention_mask(self, token_ids, valid_length):
#         attention_mask = torch.zeros_like(token_ids)
#         for i, v in enumerate(valid_length):
#             attention_mask[i][:v] = 1
#         return attention_mask.float()

#     def forward(self, token_ids, valid_length, segment_ids):
#         attention_mask = self.gen_attention_mask(token_ids, valid_length)

#         _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
#         if self.dr_rate:
#             out = self.dropout(pooler)
#         return self.classifier(out)

In [ ]:
# #BERT 모델 불러오기
# model = UmlsBERTClassifier(model,  dr_rate=0.5).to(device)

# #optimizer와 schedule 설정
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

# t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# #정확도 측정을 위한 함수 정의
# def calc_accuracy(X,Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
#     return train_acc

# train_dataloader

In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.BCELoss()
itr = 1
p_itr = 2
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

In [ ]:
# train
model.train()

for epoch in range(epochs):
    for record, diagcd in train_loader:
        optimizer.zero_grad()
        # print("record: ", record)
        # print("diagcd: ", diagcd)
        encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
        encoded_record = encoded_record.to(device)
        # print(type(encoded_record))
        diagcd = torch.tensor(diagcd).long().to(device)
        # diagcd = torch.tensor(list(diagcd), dim=0).to(device)

        outputs = model(**encoded_record, labels=diagcd)

        loss = outputs.loss
        logits = outputs.logits

        # print('lsos : ', loss)
        # print('record : ', record)
        print('encoded_record : ', encoded_record)
        print('diagcd : ', diagcd)
        print('logits : ', logits)

        predict = torch.argmax(F.softmax(logits), dim=0)
        print('predict : ', predict)
        print('predict sum: ', predict.sum())
        correct = predict.eq(diagcd)
        print('correct : ', correct)
        total_correct += correct.sum().item()
        total_len += len(diagcd)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr += 1

C:\Users\KLab\AppData\Local\Temp\ipykernel_29896\3858523646.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  diagcd = torch.tensor(diagcd).long().to(device)


encoded_record :  {'input_ids': tensor([[    2, 18846,  1828,  ...,     0,     0,     0],
        [    2,  3326,    17,  ...,     0,     0,     0],
        [    2, 11016, 10290,  ...,     0,     0,     0],
        ...,
        [    2,  2784, 10281,  ...,     0,     0,     0],
        [    2,  1725,  3517,  ...,     0,     0,     0],
        [    2, 21047,    15,  ...,     0,     0,     0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
diagcd :  tensor([ 5,  9,  7,  4,  6,  5,  7,  5,  9,  5,  3,  8,  1, 12,  5,  5

C:\Users\KLab\AppData\Local\Temp\ipykernel_29896\3858523646.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predict = torch.argmax(F.softmax(logits), dim=0)


RuntimeError: The size of tensor a (14) must match the size of tensor b (32) at non-singleton dimension 0

                                              transstr  encoded_ICD10 ICD10
278  114-69-88 2 years ago Urology and Browno Holy ...              9  R311
690  97-53-112 Diabetes and Hypertension-more than ...              5  N183
108  145-75-85 2023.4.Microscopic Hematuria 2022.10...              9  R311
499  2023/04 I heard that there is proteinuria duri...              9  R311
755  The patient is a patient of 77 years old, curr...              5  N183
..                                                 ...            ...   ...
228  The patient is a diagnostic name [Paralysis of...              5  N183
151  # HTN (2022.)2023. CR 1.24 EGFR 41 EGFR K 5.7 ...              5  N183
129  The patient is a diagnostic [DJD] patient of 8...              5  N183
781  We will ask for consultation.The patient is a ...              9  R311
159  20 years ago, DM, HTN diagnosis 10 years ago, ...              5  N183

[83 rows x 3 columns]


In [ ]:
# evaluation
model.eval()

total_len = 0
total_correct = 0

for record, diagcd in test_loader:

    encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
    encoded_record, diagcd = encoded_record.to(device), diagcd.long().to(device)

    outputs = model(**encoded_record, labels=diagcd)

    logits = outputs.logits

    predict = torch.argmax(F.softmax(logits), dim=1)
    correct = predict.eq(diagcd)

    total_correct += correct.sum().item()
    total_len += len(diagcd)

print('Test accuracy: ', total_correct / total_len)

C:\Users\KLab\AppData\Local\Temp\ipykernel_88648\1370849160.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predict = torch.argmax(F.softmax(logits), dim=1)


Test accuracy:  0.0963855421686747
